Installing necessary packages:
```Julia
using Pkg

Pkg.add(["Random", "Distributions", "DataFrames", "GLM", "Statistics", "ProgressMeter", "Plots", "Turing])
Pkg.add(url = "https://github.com/ncn-foreigners/UnobservedCountEstimation.jl")
```

In [1]:
using Turing
using Copulas

In [2]:
using Random, Distributions, DataFrames, GLM, Statistics, ProgressMeter, CSV, StatsBase, StatsPlots

In [3]:
α, β = rand(MvNormal([1.1, 2.4], .3 * [1 -.5; -.5 1]))
Q = 20

N_distr = Poisson(6000)
M_distr = (N, α) -> Poisson.(N .^ α)
u_distr = (n, N) -> Beta(n, N - n)
#u_distr = (M, μ) -> Gamma(ϕ, 1 / ϕ)

prog = Progress(10*Threads.nthreads(), "Simulation progress ...")

Threads.@threads for i in 1:(5*Threads.nthreads())
    print(string(Threads.threadid()) * " ")
    next!(prog)
end # end for

1 4 2 3 1 

Simulation progress ...   5%|█▍                          |  ETA: 0:00:13Simulation progress ...   5%|█▍                          |  ETA: 0:00:12

3 2 4 3 1 2 2 3 4 2 3 4 2 3 2 

In [5]:
α, β

(0.6918218110435266, 2.7033478193913116)

Testing `Trung.jl`

In [6]:
N = rand.(N_distr, Q)
M = reduce(vcat, rand.(M_distr(N, α), 1))
n = reduce(vcat, rand.(Binomial.(N, .04), 1))

p = (N .^ α) .* ((n ./ N) .^ β)
p = 1 ./ (1 .+ exp.(-p))

u = 1.0
m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

[m M n N p]

20×5 Matrix{Float64}:
 215.0  404.0  252.0  6132.0  0.518651
 194.0  368.0  226.0  5939.0  0.514821
 213.0  403.0  226.0  5986.0  0.514588
 227.0  422.0  245.0  5939.0  0.518432
 233.0  426.0  241.0  5954.0  0.517541
 198.0  431.0  251.0  5923.0  0.519783
 199.0  394.0  243.0  5892.0  0.518318
 201.0  398.0  255.0  5959.0  0.520396
 209.0  381.0  241.0  6031.0  0.517094
 197.0  400.0  250.0  5994.0  0.519108
 217.0  427.0  204.0  5909.0  0.511353
 207.0  423.0  254.0  6050.0  0.519576
 220.0  415.0  253.0  5888.0  0.520454
 218.0  430.0  235.0  5860.0  0.516919
 208.0  413.0  248.0  6002.0  0.518648
 254.0  434.0  262.0  5959.0  0.521944
 187.0  387.0  250.0  5895.0  0.519758
 209.0  398.0  227.0  6075.0  0.514331
 245.0  437.0  258.0  6073.0  0.520264
 213.0  413.0  208.0  5844.0  0.512233

In [8]:
@model function binomial_model(m, n, N)
    γ ~ MvNormal([1.6, 2.4], .3 * [1 .5; .5 1])
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ exp.(-μ))
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end

chains_1 = sample(binomial_model(m, n, N), PG(100), MCMCThreads(), 500, 4)

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC ~/.julia/packages/AbstractMCMC/YrmkI/src/sample.jl:310
Sampling (1 threads): 100%|█████████████████████████████| Time: 0:14:32


Chains MCMC chain (5000×24×4 Array{Float64, 3}):

Iterations        = 1:1:5000
Number of chains  = 4
Samples per chain = 5000
Wall duration     = 1169.15 seconds
Compute duration  = 1168.38 seconds
parameters        = γ[1], γ[2], M[1], M[2], M[3], M[4], M[5], M[6], M[7], M[8], M[9], M[10], M[11], M[12], M[13], M[14], M[15], M[16], M[17], M[18], M[19], M[20]
internals         = lp, logevidence

Summary Statistics
  parameters       mean        std      mcse   ess_bulk   ess_tail      rhat   ⋯
      Symbol    Float64    Float64   Float64    Float64    Float64   Float64   ⋯

        γ[1]     0.6893     0.0112    0.0013    50.0709    45.3309    1.5643   ⋯
        γ[2]     2.5943     0.3719    0.0553    45.6772        NaN    1.6337   ⋯
        M[1]   403.4493   213.1060    3.8170    55.2711    46.7482    1.5694   ⋯
        M[2]   402.6982   206.6097    5.7675    46.9759    43.1925    1.6978   ⋯
        M[3]   395.4508   208.6618    4.7968    50.4729    45.3309    1.8560   ⋯
        M[4]   4

In [9]:
[α, β, (N .^ α)', M, describe(chains_1[1:end,:,:])[1][3:end, :mean]]

5-element Vector{Any}:
 0.6918218110435266
 2.7033478193913116
  [417.2014184137074 408.0723669419337 … 414.42019415340815 403.54528487135354]
  [404, 368, 403, 422, 426, 431, 394, 398, 381, 400, 427, 423, 415, 430, 413, 434, 387, 398, 437, 413]
  [403.44935, 402.69825, 395.45075, 414.93815, 417.1381, 396.44845, 392.00555, 408.38385, 408.2646, 399.05675, 410.527, 394.4228, 399.1308, 401.7436, 395.9955, 416.3509, 389.15575, 406.40195, 415.797, 404.0382]

In [18]:
@model function binomial_model(m, n, N, μ, Σ)
    γ ~ MvNormal(μ, Σ)
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ exp.(-μ))
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end
setprogress!(false)

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


false

In [36]:
prog = Progress(nsims, "Simulation progress ...")
res = Vector{Any}(missing, nsims)
μ = [1.5, 3.4]
Σ = .2 * [1 -.6; -.6 1]
nsims = 100

Threads.@threads for k in 1:nsims
    α, β = rand(MvNormal(μ, Σ))

    N = rand.(N_distr, Q)
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    n = reduce(vcat, rand.(Binomial.(N, .2), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ exp.(-p))

    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    est = sample(binomial_model(m, n, N, μ, Σ), PG(100), 500)
    α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
    M̂ = describe(est)[1][3:end, :mean]

    μ̂_prior = [α̂₁, β̂₁]
    Σ̂_prior = vcov(mm)

    est_2 = sample(binomial_model(m, n, N, μ̂_prior, Σ̂_prior), PG(100), 500)
    α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
    M̂_2 = describe(est_2)[1][3:end, :mean]

    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q]); sum(M̂); 
        sum(N .^ α̂₄); sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q]);
        sum(M̂_2)
    ]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Simulation progress ... 100%|████████████████████████████| Time: 5:42:53


LoadError: ArgumentError: 'Vector{Float64}' iterates 'Float64' values, which doesn't satisfy the Tables.jl `AbstractRow` interface

In [47]:
df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,1.57611e7,618.0,354958.0,9.37526e8,0,Float64
2,expected,1.57604e7,582.652,3.55792e5,9.37469e8,0,Float64
3,glm_est,1.57724e7,640.196,3.52512e5,9.37852e8,0,Float64
4,ols_est,1.57718e7,601.937,3.5178e5,9.37896e8,0,Float64
5,expected_est,2.72258e5,646.569,2.89695e5,9.24024e5,0,Float64
6,expected_est_better,1.19835e7,647.684,4.08845e5,1.61499e8,0,Float64
7,actual_est,1.19835e7,645.354,4.08984e5,1.615e8,0,Float64
8,expected_est_bad_prior,1.57714e7,500.085,3.54172e5,9.37835e8,0,Float64
9,expected_est_better_bad_prior,1.57716e7,500.913,3.54184e5,9.37835e8,0,Float64


In [51]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])        for k in 2:10],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ df_res[:, 1]) for k in 2:10],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])    for k in 2:10],
    est      = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.000116574,1.20939,0.00437977,Expected value
2,-0.21841,9256.35,0.257574,glm
3,-0.215511,9065.91,0.26007,ols
4,0.281669,1.53741e7,0.377951,expected_est
5,-3.6523,1.50211e8,3.75249,expected_est_better
6,-3.65256,1.50211e8,3.75227,actual_est
7,-0.168579,8950.56,0.226103,expected_est_bad_prior
8,-0.175592,9447.54,0.232423,expected_est_better_bad_prior
9,-0.175543,9449.05,0.232135,actual_est_bad_prior


In [52]:
DataFrame(
    bias  = [mean((df_res[:, 1] .- df_res[:, k]) ./ 1000)        for k in 2:10],
    mse   = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000) for k in 2:10],
    rmse  = [sqrt(mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)) for k in 2:10],
    var   = [var(df_res[:, k] / 1000) for k in 2:10],
    sd    = [std(df_res[:, k] / 1000) for k in 2:10],
    mae   = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ 1000)    for k in 2:10],
    est   = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.689031,37519.9,193.701,9.05793e9,95173.1,1.63098,Expected value
2,-11.3189,5.55979e6,2357.92,9.06598e9,95215.4,32.4284,glm
3,-10.6342,5.68851e6,2385.06,9.06663e9,95218.9,32.8607,ols
4,15488.9,9.20643e12,3.03421e6,44438.0,210.803,15502.9,expected_est
5,3777.58,9.01605e12,3.00267e6,5.25868e8,22931.8,20322.4,expected_est_better
6,3777.57,9.01606e12,3.00267e6,5.25871e8,22931.9,20322.5,actual_est
7,-10.2836,4.19614e6,2048.45,9.06521e9,95211.4,22.2577,expected_est_bad_prior
8,-10.4429,4.20785e6,2051.3,9.06521e9,95211.4,22.3761,expected_est_better_bad_prior
9,-10.5914,4.20476e6,2050.55,9.06516e9,95211.2,22.2999,actual_est_bad_prior


In [50]:
CSV.write(pwd() * "/data_raw/init_res.csv", df_res);

### Uknown prior

In [53]:
Q = 7 # Liczba państw

N_distr = Poisson(1500)

res = Vector{Any}(missing, nsims)
μ = [1.5, 3.4]
Σ = .2 * [1 -.6; -.6 1]
nsims = 100

prog = Progress(nsims, "Simulation progress ...")

Threads.@threads for k in 1:nsims
    α, β = rand(MvNormal(μ, Σ))

    N = rand.(N_distr, Q)
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    n = reduce(vcat, rand.(Binomial.(N, .2), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ exp.(-p))
    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))
    
    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )
    
    # TODO:: estimate Σ via one of those
    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    est = sample(binomial_model(m, n, N, μ, Σ), PG(100), 500)
    α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
    M̂ = describe(est)[1][3:end, :mean]

    μ̂_prior = [α̂₁, β̂₁]
    Σ̂_prior = vcov(mm)

    est_2 = sample(binomial_model(m, n, N, μ̂_prior, Σ̂_prior), PG(100), 500)
    α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
    M̂_2 = describe(est_2)[1][3:end, :mean]

    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q]); sum(M̂); 
        sum(N .^ α̂₄); sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q]);
        sum(M̂_2)
    ]

    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

Simulation progress ... 100%|████████████████████████████| Time: 4:51:54


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,actual,2.15606e7,371.0,528770.0,1.06463e9,0,Float64
2,expected,2.15609e7,355.536,5.27528e5,1.06465e9,0,Float64
3,glm_est,2.15765e7,97.6248,5.09981e5,1.06657e9,0,Float64
4,ols_est,2.15759e7,94.0068,5.09053e5,1.06654e9,0,Float64
5,expected_est,2.54975e5,374.828,3.26216e5,512301.0,0,Float64
6,expected_est_better,3.93233e7,375.473,2.45891e7,3.45985e8,0,Float64
7,actual_est,3.93233e7,383.914,2.4589e7,3.45986e8,0,Float64
8,expected_est_bad_prior,2.15778e7,202.92,5.21789e5,1.06665e9,0,Float64
9,expected_est_better_bad_prior,2.15786e7,216.602,5.222e5,1.06665e9,0,Float64


In [54]:
DataFrame(
    rel_bias = [mean((df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])        for k in 2:10],
    rel_mse  = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ df_res[:, 1]) for k in 2:10],
    rel_mae  = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])    for k in 2:10],
    est      = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,rel_bias,rel_mse,rel_mae,est
,Float64,Float64,Float64,String
1,0.00162797,1.06888,0.00396007,Expected value
2,-0.268251,6638.04,0.337673,glm
3,-0.257848,6271.36,0.327008,ols
4,0.376868,2.11475e7,0.4379,expected_est
5,-18.6108,1.30055e9,18.6637,expected_est_better
6,-18.6118,1.30056e9,18.6642,actual_est
7,-0.0393317,2399.75,0.131305,expected_est_bad_prior
8,-0.095784,4538.62,0.1828,expected_est_better_bad_prior
9,-0.0953988,4533.34,0.183023,actual_est_bad_prior


In [55]:
DataFrame(
    bias  = [mean((df_res[:, 1] .- df_res[:, k]) ./ 1000)        for k in 2:10],
    mse   = [mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000) for k in 2:10],
    rmse  = [sqrt(mean(((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)) for k in 2:10],
    var   = [var(df_res[:, k] / 1000) for k in 2:10],
    sd    = [std(df_res[:, k] / 1000) for k in 2:10],
    mae   = [mean(abs.(df_res[:, 1] .- df_res[:, k]) ./ 1000)    for k in 2:10],
    est   = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
        "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,-0.269928,19961.3,141.285,1.26127e10,1.12306e5,1.69467,Expected value
2,-15.8847,5.35358e7,7316.82,1.26532e10,1.12486e5,71.8523,glm
3,-15.2771,5.20521e7,7214.71,1.26525e10,1.12483e5,71.4522,ols
4,21305.7,1.29334e13,3.5963e6,31108.9,176.377,21313.9,expected_est
5,-17762.7,1.05269e13,3.24452e6,3.65892e9,60489.0,40440.0,expected_est_better
6,-17762.7,1.05269e13,3.24452e6,3.65892e9,60489.0,40440.0,actual_est
7,-17.202,5.65178e7,7517.84,1.26551e10,1.12495e5,63.4928,expected_est_bad_prior
8,-17.9886,5.65988e7,7523.22,1.26551e10,1.12495e5,63.8227,expected_est_better_bad_prior
9,-18.0934,5.65669e7,7521.1,1.26551e10,1.12495e5,63.8077,actual_est_bad_prior


In [56]:
CSV.write(pwd() * "/data_raw/init_res_1.csv", df_res);

## Multivariate lognormal prior

In [222]:
@model function binomial_model_log_nor(m, n, μ, Σ)
    γ ~ MvLogNormal(μ, Σ)
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ 1 ./ μ)
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end

binomial_model_log_nor (generic function with 6 methods)

In [216]:
μ = [log(sqrt(2)), 1]
Σ = .6 * [1 .4; .4 1]

α, β = rand(MvLogNormal(μ, Σ))
Q = 20

N_distr = Poisson.(rand(DiscreteUniform(35, 1000), Q))
M_distr = (N, α) -> Poisson.(N .^ α)
u_distr = (n, N) -> Beta(n, N - n)
#u_distr = (M, μ) -> Gamma(ϕ, 1 / ϕ)

N = reduce(vcat, rand.(N_distr, 1))
M = reduce(vcat, rand.(M_distr(N, α), 1))
n = reduce(vcat, rand.(Binomial.(N, .2), 1))

p = (N .^ α) .* ((n ./ N) .^ β)
#p = 1 ./ (1 .+ exp.(-p))
p = 1 ./ (1 .+ 1 ./ p)

m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

[m M n N p]

20×5 Matrix{Float64}:
  66.0   67.0   50.0  221.0  0.968399
  34.0   34.0   30.0  117.0  0.95238
 101.0  104.0   99.0  500.0  0.981713
  54.0   54.0   45.0  188.0  0.965293
 163.0  166.0  139.0  730.0  0.985988
 140.0  142.0  147.0  704.0  0.986203
 191.0  194.0  148.0  899.0  0.987185
  47.0   48.0   30.0  181.0  0.95785
 128.0  131.0  134.0  672.0  0.985403
  36.0   38.0   33.0  140.0  0.956609
  43.0   44.0   30.0  149.0  0.955489
 112.0  114.0   77.0  432.0  0.978584
  21.0   25.0   14.0   74.0  0.924333
  68.0   70.0   64.0  281.0  0.973635
 102.0  105.0   75.0  410.0  0.977996
 124.0  125.0  107.0  561.0  0.982933
 129.0  129.0  166.0  755.0  0.987222
 140.0  141.0  134.0  640.0  0.985196
  99.0  102.0   52.0  407.0  0.973908
  29.0   36.0   21.0  153.0  0.948157

In [217]:
α,β

(0.7636366973909048, 0.47087941647000187)

In [221]:
test_1 = sample(
    binomial_model_log_nor(m / 100, n, N, μ, Σ), 
    #Gibbs(PG(100, :M), HMC(0.00001, 1000, :γ)), 
    Gibbs(PG(100, :M), HMCDA(10000, 0.65, .03, :γ)), 
    500#, initial_parametrs = reduce(vcat, [m / 100, μ])
)

┌ Warning: failed to find valid initial parameters in 10 tries; consider providing explicit initial parameters using the `initial_params` keyword
└ @ Turing.Inference ~/.julia/packages/Turing/lkUBK/src/mcmc/hmc.jl:171


LoadError: InexactError: trunc(Int64, 4.519128232039881e19)

In [210]:
describe(test_1)[1][:,:rhat]'

1×22 adjoint(::Vector{Float64}) with eltype Float64:
 2.0926  1.15072  1.4722  1.4722  …  1.46195  1.47954  1.46031  1.46195

In [211]:
describe(test_1)[1][:,:mean]'

1×22 adjoint(::Vector{Float64}) with eltype Float64:
 2.13157  12.9811  34143.3  9.34337e5  …  6.86742e5  1.12716e6  2.26363e6

In [212]:
[α, β, (N .^ α)', M, describe(test_1[1:end,:,:])[1][3:end, :mean]]

5-element Vector{Any}:
 1.280485598459593
 1.366157108903552
  [534.3941686884311 3865.0677069815115 … 4324.249529199947 6570.45731308071]
  [548, 3830, 1121, 502, 6101, 4851, 5495, 756, 2892, 2842, 6491, 1613, 2603, 5883, 708, 1650, 4169, 3189, 4292, 6598]
  [34143.318, 934336.572, 120300.652, 32087.72, 2.011325766e6, 1.335538354e6, 1.681644638e6, 57521.202, 554500.49, 564409.96, 2.180373526e6, 223670.294, 493871.536, 1.912310998e6, 56201.544, 223143.556, 1.0660661e6, 686741.54, 1.127163536e6, 2.263628594e6]

In [23]:
chains_1 = sample(
    binomial_model_log_nor(m, n, N, μ, Σ), 
    Gibbs(PG(100, :M), HMC(0.00001, 1000, :γ)), 
    500, MCMCThreads(), initial_parametrs = reduce(vcat, [M, μ]), 4
)

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC ~/.julia/packages/AbstractMCMC/YrmkI/src/sample.jl:310


counter=137
tf=TapedFunction:
* .func => binomial_model
* .ir   =>
------------------
CodeInfo(
1 ──        (m@_42 = m@_5)::Vector{Int64}
│           (μ@_43 = μ@_8)::Vector{Float64}
│           (__varinfo__@_41 = __varinfo__@_3)::DynamicPPL.TypedVarInfo{@NamedTuple{γ::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:γ, Setfield.IdentityLens}, Int64}, Vector{MvLogNormal{Float64, PDMats.PDMat{Float64, Matrix{Float64}}, Vector{Float64}}}, Vector{AbstractPPL.VarName{:γ, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, M::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:M, Setfield.IndexLens{Tuple{Int64}}}, Int64}, Vector{Poisson{Float64}}, Vector{AbstractPPL.VarName{:M, Setfield.IndexLens{Tuple{Int64}}}}, Vector{Int64}, Vector{Set{DynamicPPL.Selector}}}}, Float64}
│           Core.NewvarNode(:(@_10))::Any
│           Core.NewvarNode(:(@_11))::Any
│           Core.NewvarNode(:(@_12))::Any
│           Core.NewvarNode(:(@_13))::Any
│           Core.NewvarNode(:(retval#27

LoadError: TaskFailedException

[91m    nested task error: [39mTaskFailedException
    
    [91m    nested task error: [39mInexactError: trunc(Int64, 1.893205781989023e19)
        Stacktrace:
          [1] [0m[1mconsume[22m[0m[1m([22m[90mttask[39m::[0mTapedTask[90m{typeof(binomial_model), Tuple{DynamicPPL.Model{typeof(binomial_model), (:m, :n, :N, :μ, :Σ), (), (), Tuple{Vector{Int64}, Vector{Int64}, Vector{Int64}, Vector{Float64}, Matrix{Float64}}, Tuple{}, DynamicPPL.DefaultContext}, DynamicPPL.TypedVarInfo{@NamedTuple{γ::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:γ, Setfield.IdentityLens}, Int64}, Vector{MvLogNormal{Float64, PDMats.PDMat{Float64, Matrix{Float64}}, Vector{Float64}}}, Vector{AbstractPPL.VarName{:γ, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, M::DynamicPPL.Metadata{Dict{AbstractPPL.VarName{:M, Setfield.IndexLens{Tuple{Int64}}}, Int64}, Vector{Poisson{Float64}}, Vector{AbstractPPL.VarName{:M, Setfield.IndexLens{Tuple{Int6

In [ ]:
prog = Progress(nsims, "Simulation progress ...")
res = Vector{Any}(missing, nsims)
μ = [1.5, 3.4]
Σ = .2 * [1 -.6; -.6 1]
nsims = 100

Threads.@threads for k in 1:nsims
    α, β = rand(MvNormal(μ, Σ))

    N = rand.(N_distr, Q)
    M = reduce(vcat, rand.(M_distr(N, α), 1))
    n = reduce(vcat, rand.(Binomial.(N, .2), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ exp.(-p))

    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    α̂₁, β̂₁ = coef(mm)

    ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    α̂₂, β̂₂ = coef(ols)

    est = sample(binomial_model(m, n, N, μ, Σ), PG(100), 500)
    α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
    M̂ = describe(est)[1][3:end, :mean]

    μ̂_prior = [α̂₁, β̂₁]
    Σ̂_prior = vcov(mm)

    est_2 = sample(binomial_model(m, n, N, μ̂_prior, Σ̂_prior), PG(100), 500)
    α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
    M̂_2 = describe(est_2)[1][3:end, :mean]

    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q]); sum(M̂); 
        sum(N .^ α̂₄); sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q]);
        sum(M̂_2)
    ]
    #push!(res, [sum(M) sum(N .^ α) sum(N .^ α̂₁) sum(N .^ α̂₂)])
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est", "expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"])

describe(df_res)

## Gamma prior

In [3]:
@model function binomial_model_gamma(m, n, N, Σ, a, b)
    C  = GaussianCopula(Σ)
    d₁ = Gamma(a[1], b[1])
    d₂ = Gamma(a[2], b[2])

    γ ~ SklarDist(C, (d₁, d₂))
    #Turing.Turing.@addlogprob! loglikelihood(D, X)
    ξ = N .^ γ[1]
    μ = ξ .* ((n ./ N) .^ γ[2])
    μ = 1 ./ (1 .+ 1 ./ μ)
    M = Vector(undef, length(m))
    
    for i in eachindex(M)
        M[i] ~ Poisson(ξ[i])
    end

    for i in eachindex(m)
        m[i] ~ Binomial(M[i], μ[i])
    end
    return m
end

binomial_model_gamma (generic function with 2 methods)

In [4]:
a = [24.0, 13.5]
b = [1 / 25.0, 1 / 10.5]
Σ = [1 .5; .5 1]

C = GaussianCopula(Σ)
d = SklarDist(C, (Gamma(a[1], b[1]), Gamma(a[2], b[2])))

α, β = rand(d)

Q = 20

N_distr = Poisson.(rand(DiscreteUniform(20, 300), Q))
M_distr = (N, α) -> Poisson.(N .^ α)
u_distr = (n, N) -> Beta(n, N - n)
#u_distr = (M, μ) -> Gamma(ϕ, 1 / ϕ)

N = reduce(vcat, rand.(N_distr, 1))
M = reduce(vcat, rand.(M_distr(N, α), 1))
n = reduce(vcat, rand.(Binomial.(N, .1), 1))

p = (N .^ α) .* ((n ./ N) .^ β)
#p = 1 ./ (1 .+ exp.(-p))
p = 1 ./ (1 .+ 1 ./ p)
u = 1

m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

println([α, β])

[m M n N p]

[1.1689754889148265, 1.6108858946058267]


20×5 Matrix{Float64}:
 199.0  242.0  13.0  101.0  0.890159
 759.0  786.0  40.0  292.0  0.968743
 293.0  324.0  17.0  148.0  0.913383
 221.0  268.0   8.0  117.0  0.776483
  52.0   87.0   3.0   42.0  0.529472
 395.0  437.0  18.0  189.0  0.91211
 465.0  510.0  18.0  215.0  0.907436
 564.0  611.0  20.0  233.0  0.918108
 174.0  212.0   9.0   98.0  0.819555
 351.0  400.0  14.0  165.0  0.880255
 241.0  286.0   9.0  125.0  0.803102
 325.0  376.0  13.0  156.0  0.869921
  10.0   88.0   1.0   42.0  0.160877
 155.0  219.0   7.0   98.0  0.751848
 413.0  430.0  24.0  192.0  0.942467
 117.0  148.0   7.0   75.0  0.77324
 741.0  772.0  36.0  307.0  0.96238
  39.0   78.0   3.0   42.0  0.529472
 313.0  357.0  13.0  164.0  0.867399
 366.0  393.0  21.0  157.0  0.935231

In [5]:
test_1 = sample(
    binomial_model_gamma(m, n, N, Σ, a, b), 
    #Gibbs(PG(100, :M), HMC(0.00001, 1000, :γ)), 
    PG(50), 
    MCMCThreads(),
    1500,
    4
)

Sampling (4 threads): 100%|█████████████████████████████| Time: 0:02:02


Chains MCMC chain (1500×24×4 Array{Float64, 3}):

Iterations        = 1:1:1500
Number of chains  = 4
Samples per chain = 1500
Wall duration     = 296.36 seconds
Compute duration  = 817.04 seconds
parameters        = γ[1], γ[2], M[1], M[2], M[3], M[4], M[5], M[6], M[7], M[8], M[9], M[10], M[11], M[12], M[13], M[14], M[15], M[16], M[17], M[18], M[19], M[20]
internals         = lp, logevidence

Summary Statistics
  parameters       mean       std      mcse   ess_bulk   ess_tail      rhat    ⋯
      Symbol    Float64   Float64   Float64    Float64    Float64   Float64    ⋯

        γ[1]     1.1748    0.0046    0.0010    22.3652    27.9967    1.6929    ⋯
        γ[2]     1.7031    0.0736    0.0144    20.8981    35.9394    1.5109    ⋯
        M[1]   231.2730    7.8151    2.0038    15.7069        NaN    1.7941    ⋯
        M[2]   792.3297   13.1069    3.0574    16.0581    16.2396    1.8106    ⋯
        M[3]   345.9452   21.3263    5.1736    16.5932    14.8062    1.5848    ⋯
        M[4]   272

In [6]:
minimum(describe(test_1)[1][:,:rhat]), maximum(describe(test_1)[1][:,:rhat])

(1.3630617885421306, 3.1325345454150413)

In [7]:
[describe(test_1)[1][:,:rhat]', describe(test_1)[1][:,:mean]']

2-element Vector{LinearAlgebra.Adjoint{Float64, Vector{Float64}}}:
 [1.6929109314353759 1.510930318205592 … 1.5146245695825467 1.3630617885421306]
 [1.1748142589473187 1.7031221172461446 … 393.151 392.337]

In [8]:
df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(y)) ~ 0 + x1 + x2

Coefficients:
──────────────────────────────────────────────────────────────
      Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────
x1  1.45553   0.0423466  34.37    <1e-17   1.36656    1.54449
x2  0.68288   0.0835333   8.17    <1e-06   0.507383   0.858377
──────────────────────────────────────────────────────────────

In [9]:
    Σ̂_prior = vcov(mm)
    b_prior = [coef(mm)[1] / Σ̂_prior[1, 1], coef(mm)[2] / Σ̂_prior[2, 2]] .^ -1
    a_prior = coef(mm) ./ b_prior

2-element Vector{Float64}:
 7791.758492282519
   87.06786930417682

In [10]:
test_2 = sample(
    binomial_model_gamma(m, n, N, Σ̂_prior, a_prior, b_prior), 
    #Gibbs(PG(100, :M), HMC(0.00001, 1000, :γ)), 
    PG(50), 
    MCMCThreads(),
    1500,
    4
)

Sampling (4 threads): 100%|█████████████████████████████| Time: 0:01:57


Chains MCMC chain (1500×24×4 Array{Float64, 3}):

Iterations        = 1:1:1500
Number of chains  = 4
Samples per chain = 1500
Wall duration     = 288.82 seconds
Compute duration  = 805.37 seconds
parameters        = γ[1], γ[2], M[1], M[2], M[3], M[4], M[5], M[6], M[7], M[8], M[9], M[10], M[11], M[12], M[13], M[14], M[15], M[16], M[17], M[18], M[19], M[20]
internals         = lp, logevidence

Summary Statistics
  parameters        mean       std      mcse   ess_bulk   ess_tail      rhat   ⋯
      Symbol     Float64   Float64   Float64    Float64    Float64   Float64   ⋯

        γ[1]      1.2267    0.0049    0.0011    16.6461    14.9420    1.5428   ⋯
        γ[2]      0.1959    0.0093    0.0018    20.3585    22.1738    1.3606   ⋯
        M[1]    270.1220    9.5962    2.3149    16.5856    19.1966    1.5628   ⋯
        M[2]   1062.1675   27.8343    6.0233    18.8074    17.1570    1.4337   ⋯
        M[3]    453.4428   15.4553    3.3358    18.6905    16.2273    1.3221   ⋯
        M[4]    34

In [11]:
minimum(describe(test_2)[1][:,:rhat]), maximum(describe(test_2)[1][:,:rhat])

(1.3220828357620587, 2.9075242925412974)

In [12]:
[describe(test_2)[1][:,:rhat]', describe(test_2)[1][:,:mean]']

2-element Vector{LinearAlgebra.Adjoint{Float64, Vector{Float64}}}:
 [1.5427720755349703 1.360560871828682 … 1.5025631374199826 2.9075242925412974]
 [1.2267000289076415 0.19588102486744965 … 530.4901666666667 504.09233333333333]

In [13]:
[α, β, (N .^ α)', M, 
 describe(test_1[1:end,:,:])[1][3:end, :mean],
 describe(test_2[1:end,:,:])[1][3:end, :mean],
 (N .^ coef(mm)[1])', (N .^ coef(ols)[1])']

8-element Vector{Any}:
 1.1689754889148265
 1.6108858946058267
  [220.2939437646305 762.030096370328 … 388.2381153979856 368.937558801142]
  [242, 786, 324, 268, 87, 437, 510, 611, 212, 400, 286, 376, 88, 219, 430, 148, 772, 78, 357, 393]
  [231.273, 792.3296666666666, 345.9451666666667, 272.18483333333336, 79.82016666666667, 463.7835, 542.6993333333334, 624.3885, 212.65916666666666, 408.9028333333333, 300.36516666666665, 376.9745, 83.233, 208.01633333333334, 469.0375, 158.61783333333332, 810.0916666666667, 76.63833333333334, 393.151, 392.337]
  [270.122, 1062.1675, 453.44283333333334, 345.46883333333335, 104.47233333333334, 612.5523333333333, 713.9773333333334, 782.528, 273.6231666666667, 531.6895, 376.22583333333336, 485.54516666666666, 96.38083333333333, 279.3666666666667, 634.9696666666666, 196.00066666666666, 1122.6613333333332, 102.48716666666667, 530.4901666666667, 504.09233333333333]
  [368.19712199270793 1433.3933212685752 … 684.8679197488841 647.668930645026]
  [826.691533162

In [14]:
rr = DataFrame([
    (N .^ coef(mm)[1])' - M' 
    (N .^ coef(ols)[1])' - M'
    describe(test_1)[1][3:end, :mean]' - M'
    describe(test_2)[1][3:end, :mean]' - M'
]', :auto)

describe(rr)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,x1,276.032,31.7302,260.531,756.334,0,Float64
2,x2,1208.99,142.506,1147.88,3397.62,0,Float64
3,x3,10.9224,-10.9837,7.61625,39.0375,0,Float64
4,x4,122.713,8.38083,110.319,350.661,0,Float64


In [15]:
setprogress!(false)
# simulation parameters
chain_sim = 500 # nie patrzyłem czy warto to wyżej ustawić
nsims = 500
pg_sim = 25 # na symulacje wystardczy bo dla podwojonej wartości poprawy dużej nie ma

#
prog = Progress(nsims, "Simulation progress ...")
res = Vector{Any}(missing, nsims)

# hyper parameters
a = [24.75, 16.5]
b = [1 / 25.0, 1 / 13.5]
Q = 10
Σ = [1 .25; .25 1]
C = GaussianCopula(Σ)
d = SklarDist(C, (Gamma(a[1], b[1]), Gamma(a[2], b[2])))

N_distr = Poisson.(rand(DiscreteUniform(50, 500), Q))
M_distr = (N, α) -> Poisson.(N .^ α)
#u_distr = (n, N) -> Beta(n, N - n)

Threads.@threads for k in 1:nsims
    α, β = rand(d)

    N = reduce(vcat, rand.(N_distr, 1))
    n = reduce(vcat, rand.(Binomial.(N, .1), 1))
    
    M = reduce(vcat, rand.(M_distr(N, α), 1))

    p = (N .^ α) .* ((n ./ N) .^ β)
    p = 1 ./ (1 .+ 1 ./ p)

    #u = reduce(vcat, rand.(u_distr.(n, N), 1))
    u = 1.0
    m = reduce(vcat, rand.(Binomial.(M, p * u), 1))

    df1 = DataFrame(
        y = m,
        x1 = log.(N),
        x2 = log.(n ./ N)
    )

    α̂₁, β̂₁ = [NaN, NaN]
    α̂₂, β̂₂ = [NaN, NaN]
    mm = nothing
    ols = nothing

    try
        mm = glm(@formula(y ~ x1 + x2 + 0), df1, Poisson(), LogLink())
    catch e
        println(e)
    end # en try

    try
        ols = lm(@formula(log(y) ~ x1 + x2 + 0), df1)
    catch e
        println(e)
    end # en try

    if !isnothing(mm)
        α̂₁, β̂₁ = coef(mm)
    end # end if

    if !isnothing(ols)
        α̂₂, β̂₂ = coef(ols)
    end # end if
    
    α̂₃, β̂₃ = [NaN, NaN]
    M̂ = [NaN for ii in 1:Q]
    est_M = [NaN for ii in 1:Q]
    est = nothing
    
    try
        est = sample(binomial_model_gamma(m, n, N, Σ, a, b), PG(pg_sim), chain_sim)
    catch e
        println(e)
    end # end try

    if !isnothing(est)
        α̂₃, β̂₃ = describe(est)[1][1:2, :mean]
        M̂      = describe(est)[1][3:end, :mean]
        est_M  = sum([mean(N[x] .^ est[:, 1, :]) for x in 1:Q])
    end # end if

    α̂₄, β̂₄ = [NaN, NaN]
    M̂_2 = [NaN for ii in 1:Q]
    est_M_2 = [NaN for ii in 1:Q]
    est_2 = nothing

    # estimated priors
    if !isnothing(mm)
        μ̂_prior = [α̂₁, β̂₁]
        Σ̂_prior = vcov(mm)
        b_prior = [α̂₁ / Σ̂_prior[1, 1], β̂₁ / Σ̂_prior[2, 2]] .^ -1
        a_prior = [α̂₁, β̂₁] ./ b_prior
    end # end if

    try
        est_2 = sample(binomial_model_gamma(m, n, N, Σ̂_prior, a_prior, b_prior), PG(pg_sim), chain_sim)
    catch e
        println(e)
    end # end try

    if !isnothing(est_2)
        α̂₄, β̂₄ = describe(est_2)[1][1:2, :mean]
        M̂_2    = describe(est_2)[1][3:end, :mean]
        est_M_2  = sum([mean(N[x] .^ est_2[:, 1, :]) for x in 1:Q])
    end # end if

    # this fails if nothing is passed to get index
    res[k] = [
        sum(M); sum(N .^ α); sum(N .^ α̂₁); sum(N .^ α̂₂); sum(N .^ α̂₃);
        sum(est_M); sum(M̂); sum(N .^ α̂₄); sum(est_M_2); sum(M̂_2)
    ]
    
    next!(prog)
end # end for

df_res = DataFrame(mapreduce(permutedims, vcat, res), 
    ["actual", "expected", "glm_est", "ols_est", "expected_est" , "expected_est_better", 
        "actual_est","expected_est_bad_prior" , "expected_est_better_bad_prior", "actual_est_bad_prior"
    ])

describe(df_res)

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


DomainError(-0.14403128747896488, "Gamma: the condition θ > zero(θ) is not satisfied.")


Simulation progress ...   1%|▍                           |  ETA: 4:46:55

DomainError(-0.2255992778672261, "Gamma: the condition θ > zero(θ) is not satisfied.")


Simulation progress ...   4%|█▏                          |  ETA: 3:19:14

...

DomainError(-0.4162656636860542, "Gamma: the condition θ > zero(θ) is not satisfied.")


Simulation progress ...  90%|█████████████████████████▏  |  ETA: 0:38:42

DomainError(-5.510855429267145, "Gamma: the condition θ > zero(θ) is not satisfied.")


Simulation progress ... 100%|████████████████████████████| Time: 6:02:45


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Union…,Float64,Int64,DataType
1,actual,5843.86,188.0,2257.5,109289.0,0,Float64
2,expected,5849.84,173.649,2277.35,1.09283e5,0,Float64
3,glm_est,NaN,NaN,,NaN,0,Float64
4,ols_est,NaN,NaN,,NaN,0,Float64
5,expected_est,6097.22,178.736,2361.75,1.17721e5,0,Float64
6,expected_est_better,6144.98,182.157,2363.52,1.17923e5,0,Float64
7,actual_est,6149.57,194.506,2402.03,1.18056e5,0,Float64
8,expected_est_bad_prior,NaN,NaN,,NaN,0,Float64
9,expected_est_better_bad_prior,NaN,NaN,,NaN,0,Float64


In [16]:
DataFrame(
    rel_bias = [mean(((df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])[.!isnan.(df_res[:, k])])        for k in 2:10],
    rel_mse  = [mean((((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ df_res[:, 1])[.!isnan.(df_res[:, k])]) for k in 2:10],
    rel_mae  = [mean((abs.(df_res[:, 1] .- df_res[:, k]) ./ df_res[:, 1])[.!isnan.(df_res[:, k])])    for k in 2:10],
    NaN_proc = [mean(isnan.(df_res[:, k]))                                                            for k in 2:10],
    est      = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est", 
                "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"]
)

Row,rel_bias,rel_mse,rel_mae,NaN_proc,est
,Float64,Float64,Float64,Float64,String
1,-0.00140554,0.98961,0.0178622,0.0,Expected value
2,-0.934158,4990.58,0.946013,0.002,glm
3,-1.30383,17617.6,1.31641,0.006,ols
4,-0.0536558,21.2389,0.0639445,0.0,expected_est
5,-0.0556776,24.7512,0.0653268,0.0,expected_est_better
6,-0.0575094,24.6404,0.0651531,0.0,actual_est
7,-0.138514,451.544,0.157831,0.052,expected_est_bad_prior
8,-0.140134,455.753,0.158949,0.052,expected_est_better_bad_prior
9,-0.135704,447.453,0.155855,0.052,actual_est_bad_prior


In [18]:
DataFrame(
    bias  = [mean(((df_res[:, 1] .- df_res[:, k]) ./ 1000)[.!isnan.(df_res[:, k])])              for k in 2:10],
    mse   = [mean((((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)[.!isnan.(df_res[:, k])])       for k in 2:10],
    rmse  = [sqrt(mean((((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)[.!isnan.(df_res[:, k])])) for k in 2:10],
    var   = [var((df_res[:, k] / 1000)[.!isnan.(df_res[:, k])])                                  for k in 2:10],
    sd    = [std((df_res[:, k] / 1000)[.!isnan.(df_res[:, k])])                                  for k in 2:10],
    mae   = [mean((abs.(df_res[:, 1] .- df_res[:, k]) ./ 1000)[.!isnan.(df_res[:, k])])          for k in 2:10],
    est   = ["Expected value", "glm", "ols", "expected_est", "expected_est_better", "actual_est"
        , "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"
    ]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,-0.00597458,5.03973,2.24493,135.377,11.6352,0.0475442,Expected value
2,-2.28064,17688.4,132.998,169.75,13.0288,2.31801,glm
3,-3.63036,77796.3,278.92,239.26,15.468,3.67925,ols
4,-0.253359,596.127,24.4157,147.589,12.1486,0.29334,expected_est
5,-0.301117,760.764,27.582,152.808,12.3615,0.323707,expected_est_better
6,-0.305705,766.102,27.6786,152.888,12.3648,0.324931,actual_est
7,-0.519107,3092.5,55.6102,148.168,12.1724,0.557509,expected_est_bad_prior
8,-0.522229,3107.83,55.7479,148.177,12.1728,0.559964,expected_est_better_bad_prior
9,-0.505559,3038.18,55.1197,147.634,12.1505,0.545699,actual_est_bad_prior


In [19]:
DataFrame(
    bias  = [mean(((df_res[:, 2] .- df_res[:, k]) ./ 1000)[.!isnan.(df_res[:, k])])              for k in 3:10],
    mse   = [mean((((df_res[:, 2] .- df_res[:, k]) .^ 2) ./ 1000)[.!isnan.(df_res[:, k])])       for k in 3:10],
    rmse  = [sqrt(mean((((df_res[:, 1] .- df_res[:, k]) .^ 2) ./ 1000)[.!isnan.(df_res[:, k])])) for k in 3:10],
    var   = [var((df_res[:, k] / 1000)[.!isnan.(df_res[:, k])])                                  for k in 3:10],
    sd    = [std((df_res[:, k] / 1000)[.!isnan.(df_res[:, k])])                                  for k in 3:10],
    mae   = [mean((abs.(df_res[:, 2] .- df_res[:, k]) ./ 1000)[.!isnan.(df_res[:, k])])          for k in 3:10],
    est   = ["glm", "ols", "expected_est", "expected_est_better", "actual_est"
        , "expected_est_bad_prior", "expected_est_better_bad_prior", "actual_est_bad_prior"
    ]
)

Row,bias,mse,rmse,var,sd,mae,est
,Float64,Float64,Float64,Float64,Float64,Float64,String
1,-2.2746,17623.8,132.998,169.75,13.0288,2.3142,glm
2,-3.62427,77636.3,278.92,239.26,15.468,3.67393,ols
3,-0.247384,598.402,24.4157,147.589,12.1486,0.292371,expected_est
4,-0.295142,757.102,27.582,152.808,12.3615,0.322072,expected_est_better
5,-0.299731,762.273,27.6786,152.888,12.3648,0.322613,actual_est
6,-0.514244,3071.87,55.6102,148.168,12.1724,0.558815,expected_est_bad_prior
7,-0.517365,3087.15,55.7479,148.177,12.1728,0.561236,expected_est_better_bad_prior
8,-0.500695,3017.65,55.1197,147.634,12.1505,0.546234,actual_est_bad_prior


In [20]:
CSV.write(pwd() * "/data_raw/gamma_res.csv", df_res);